In [2]:
# Mount the Google Drive so we can access data
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -r '/content/drive/My Drive/Colab Notebooks/MRI project/fastMRI-GAN-master/requirements.txt'
!pip install s3fs
#!pip install pydicom
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (10, 10)
import h5py
import s3fs
import pydicom
from pydicom.data import get_testdata_file

In [4]:
!pip install fastmri

In [5]:
import sys
sys.path.append('drive/My Drive/Colab Notebooks/MRI project/fastMRI-GAN-master/')
!ls 'drive/My Drive/Colab Notebooks/MRI project/fastMRI-GAN-master'
!ls

data		   jade_job_big.sh     logs	  requirements.txt
fastMRIdata.ipynb  jade_job_small2.sh  model	  trained_models
jade_job_big2.sh   jade_job_small.sh   README.md
drive  sample_data


In [6]:
s3 = s3fs.S3FileSystem(anon=True)
s3.ls('s3://cs7643-fastmri/data/')

['cs7643-fastmri/data/',
 'cs7643-fastmri/data/fastMRI_brain_DICOM',
 'cs7643-fastmri/data/multicoil_challenge',
 'cs7643-fastmri/data/multicoil_challenge_transfer',
 'cs7643-fastmri/data/multicoil_test',
 'cs7643-fastmri/data/multicoil_train',
 'cs7643-fastmri/data/multicoil_val']

In [28]:
%matplotlib inline


import h5py
import numpy as np
import pathlib
from model.utils.subsample import MaskFunc
from fastmri.data.subsample import RandomMaskFunc
import model.utils.transforms as Tlocal
import fastmri
from fastmri.data import transforms as T
from matplotlib import pyplot as plt

def show_slices(data, slice_nums, cmap=None):
    fig = plt.figure()
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)

def get_training_pair(file, centre_fraction, acceleration):

    """


    :param file: The training image
    :param centre_fraction: randomly generated centre fraction
    :param acceleration: randomly generated
    :return:
    """


    hf = h5py.File(s3.open(file, 'rb'))

    volume_kspace = hf['kspace'][()]
    volume_image = hf['reconstruction_rss'][()]
    mask_func = RandomMaskFunc(center_fractions=[centre_fraction], accelerations=[acceleration])  # Create the mask function object

    volume_kspace_tensor = T.to_tensor(volume_kspace)
    masked_kspace, mask = T.apply_mask(volume_kspace_tensor, mask_func)
    masked_kspace_np=masked_kspace.numpy().reshape(masked_kspace.shape)

    return np.expand_dims(volume_image,3), masked_kspace_np


def get_training_pair_images(file, centre_fraction, acceleration, image_size=256):
    """
    :param file: The training image
    :param centre_fraction: randomly generated centre fraction
    :param acceleration: randomly generated
    :return: true gold standard and the fft of the masked k-space image
    """
    
    hf = h5py.File(s3.open(file, 'rb'))
    volume_kspace = hf['kspace'][()]
    volume_image = hf['reconstruction_rss'][()]
    mask_func = MaskFunc(center_fractions=[centre_fraction], accelerations=[acceleration])  # Create the mask function object

    volume_kspace_tensor = T.to_tensor(volume_kspace)
    masked_kspace, mask =  T.apply_mask(volume_kspace_tensor, mask_func)
    ##masked_kspace_np=masked_kspace.numpy().reshape(masked_kspace.shape)
 
    volume_image=T.center_crop(volume_image, shape=[image_size,image_size])
    
    print(volume_image.shape)
    print(volume_image.shape[1])
    print(volume_image.shape[2])
    
    recon_image= fastmri.ifft2c(masked_kspace) 			 # complex image
    recon_image=T.complex_center_crop(recon_image, shape=[volume_image.shape[1],volume_image.shape[2]])

    volume_image, mean, std=T.normalize_instance(volume_image)    
    #recon_image=T.normalize_instance(recon_image)    
    
    
    recon_image_abs= fastmri.complex_abs(recon_image)                # compute absolute value to get a real image
    #recon_image_rss= T.root_sum_of_square(recon_image_abs,dim=0) # compute absolute rss
    
    recon_image_abs, mean ,std =Tlocal.normalize_instance(recon_image_abs)
        
    
    return np.expand_dims(volume_image,3), recon_image_abs.numpy()


def get_random_accelerations():

   """
      : we apply these to fully sampled k-space to obtain q
      :return:random centre_fractions between 0.1 and 0.001 and accelerations between 1 and 15
   """
   acceleration = np.random.randint(1, high=8, size=1)
   centre_fraction = np.random.uniform(0, 1, 1)
   decimal = np.random.randint(1, high=3, size=1)
   centre_fraction = centre_fraction / (10 ** decimal)

   return float(centre_fraction), float(acceleration)





In [8]:
#datadir= '/media/jehill/DATA/ML_data/fastmri/singlecoil/train/singlecoil_train/'
#files = list(pathlib.Path(datadir).iterdir())
s3 = s3fs.S3FileSystem(anon=True)
datadir= 's3://cs7643-fastmri/data/multicoil_train/'
files=s3.ls(datadir)
print(len(files))


1592


In [31]:
def train(file):
    #files = list(pathlib.Path(datadir).iterdir())
    #np.random.shuffle(files)
    #for file in files:
        print(file)
        centre_fraction,acceleration=get_random_accelerations()
        print("Centre Fraction is {}". format(centre_fraction))
        print("Acceleration {}". format(acceleration))
        print(acceleration)
        image, masked_kspace =get_training_pair_images(file, centre_fraction=centre_fraction,acceleration=acceleration)
        print(image.shape)
        print(masked_kspace.shape)

        #image = fastmri.ifft2c(image)           # Apply Inverse Fourier Transform to get the complex image
        #image_abs = fastmri.complex_abs(image)   # Compute absolute value to get a real image
        #image_rss = fastmri.rss(image_abs, dim=0)

        sampled_image = fastmri.ifft2c(image)           # Apply Inverse Fourier Transform to get the complex image
        sampled_image_abs = fastmri.complex_abs(sampled_image)   # Compute absolute value to get a real image
        sampled_image_rss = fastmri.rss(sampled_image_abs, dim=0)
        print(sampled_image.shape,sampled_image_abs.shape,sampled_image_rss.shape)
        
        plt.subplot(1,2,1)
        plt.imshow(sampled_image_rss, cmap='gray')
        plt.subplot(1,2,2)
        plt.imshow(image, cmap='gray')
        
        print(image.max())
        print(image.min())
        print(masked_kspace.min())
        print(masked_kspace.max())
                

In [10]:
s3 = s3fs.S3FileSystem(anon=True)
s3.ls('s3://cs7643-fastmri/data/multicoil_train')
#file_name = 's3://cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002425.h5'
#hf = h5py.File(file_name)

['cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002425.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002428.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002429.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002435.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002442.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002452.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002456.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002458.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002460.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002466.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002467.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002469.h5',
 'cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002474.h5',
 'cs7643-fastmri/data/multicoil_train/

In [32]:
import torch
i=0
for filename in files:
  #filename = 's3://cs7643-fastmri/data/fastMRI_brain_DICOM/100099070170/279.dcm'
  print(filename)
  data = h5py.File(s3.open(filename, 'rb'))
  print('Keys:', list(data.keys()))
  print('Attrs:', dict(data.attrs))
  #volume_kspace = data['kspace'][()]
  #volume_image = data['reconstruction_rss'][()]
  #volume_kspace = data['kspace'][()]
  #volume_image = data['reconstruction_rss'][()]
  #centre_fraction,acceleration=get_random_accelerations()
  #mask_func = MaskFunc(center_fractions=[centre_fraction], accelerations=[acceleration])  # Create the mask function object

  #volume_kspace_tensor = T.to_tensor(volume_kspace)
  #torch.fft.fft2(volume_kspace_tensor)
  train("s3://"+filename)
  if i==0:
    break
  i=i+1

#train(files[1])

cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002425.h5
Keys: ['ismrmrd_header', 'kspace', 'reconstruction_rss']
Attrs: {'acquisition': 'AXFLAIR', 'max': 0.0004274357358916608, 'norm': 0.09131797120370877, 'patient_id': 'dcfc3e70496246a709dab23831262b60589f4383d6e05bf5559731ea8d80bb1f'}
s3://cs7643-fastmri/data/multicoil_train/file_brain_AXFLAIR_200_6002425.h5
Centre Fraction is 0.005783517229853831
Acceleration 7.0
7.0
(16, 256, 256)
256
256
(16, 256, 256, 1)
(16, 20, 256, 256)


ValueError: ignored

In [ ]:
train(files[1])

In [ ]:
train(files[1])
#Lower acceleration: Note Random sampling 

In [ ]:
#Another image
train(files[100])

In [ ]:
train(files[100])

In [ ]:
train(files[89])

In [ ]:
train(files[89])

In [ ]:
import tensorflow as tf

In [ ]:
def read_training_images(file):
    #files = list(pathlib.Path(datadir).iterdir())
    #np.random.shuffle(files)
    #for file in files:
        #print(file)
        centre_fraction,acceleration=get_random_accelerations()
        image, partial_images =get_training_pair_images(file, centre_fraction=centre_fraction,acceleration=acceleration)
        return image
            